In [38]:
import pandas as pd
import sqlite3
import os

subject = 'IorNYll1lS' #! The ID of the subject on Back4app
folder_path = 'database extractor/data'
df_file = '20231205_122301_14ChannelSensorDatabase.db'
db_file_path = f'{folder_path}/{df_file}'


In [7]:

# Replace 'your_db_file.db' with the path to your database file
db_file_path = 'database extractor/data/20231205_122301_14ChannelSensorDatabase.db'

# Create a connection to the SQLite database
conn = sqlite3.connect(db_file_path)

# Write your SQL query (replace 'your_table_name' with the actual table name)
sql_query = 'SELECT * FROM USER_SESSION_TABLE'

# Use pandas to read data from the database into a DataFrame
df = pd.read_sql(sql_query, conn)

# Close the database connection
conn.close()

# Display the DataFrame
print(df)

     SESSION_ID            USERNAME DEVICE_ADDRESS_LEFT DEVICE_ADDRESS_RIGHT  \
0             1  tangtop5@yahoo.com   E1:95:F0:44:B7:3E    EC:39:5A:86:B1:49   
1             2  tangtop5@yahoo.com   E1:95:F0:44:B7:3E    EC:39:5A:86:B1:49   
2             3  tangtop5@yahoo.com   E1:95:F0:44:B7:3E    EC:39:5A:86:B1:49   
3             4  tangtop5@yahoo.com   E1:95:F0:44:B7:3E    EC:39:5A:86:B1:49   
4             5  tangtop5@yahoo.com   E1:95:F0:44:B7:3E    EC:39:5A:86:B1:49   
..          ...                 ...                 ...                  ...   
137         138  tangtop5@yahoo.com   E1:95:F0:44:B7:3E    EA:A6:84:C8:75:9C   
138         139  tangtop5@yahoo.com   E1:95:F0:44:B7:3E    EA:A6:84:C8:75:9C   
139         140  tangtop5@yahoo.com   E1:95:F0:44:B7:3E    EA:A6:84:C8:75:9C   
140         141  tangtop5@yahoo.com   E1:95:F0:44:B7:3E    EA:A6:84:C8:75:9C   
141         142  tangtop5@yahoo.com   E1:95:F0:44:B7:3E    EA:A6:84:C8:75:9C   

    START_DATE START_TIME START_BATTERY

In [4]:
# Create a connection to the SQLite database
conn = sqlite3.connect(db_file_path)

# Write SQL queries for each table
right_insole_query = 'SELECT * FROM RIGHT_INSOLE_RAW'
left_insole_query = 'SELECT * FROM LEFT_INSOLE_RAW'

# Use pandas to read data from each table into DataFrames
right_insole_df = pd.read_sql(right_insole_query, conn)
left_insole_df = pd.read_sql(left_insole_query, conn)

# Close the database connection
conn.close()

# Group the DataFrames by SESSION_ID
grouped_right_insole = right_insole_df.groupby('SESSION_ID').agg(list).reset_index()
grouped_left_insole = left_insole_df.groupby('SESSION_ID').agg(list).reset_index()

# Display the grouped DataFrames
print("Grouped RIGHT_INSOLE_RAW:")
print(grouped_right_insole)

print("\nGrouped LEFT_INSOLE_RAW:")
print(grouped_left_insole)

Grouped RIGHT_INSOLE_RAW:
     SESSION_ID                                             RAW_ID  \
0             0  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...   
1             1  [22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 3...   
2             2  [904, 905, 906, 907, 908, 909, 910, 911, 912, ...   
3             3  [937, 938, 939, 940, 941, 942, 943, 944, 945, ...   
4             4  [2620, 2621, 2622, 2623, 2624, 2625, 2626, 262...   
..          ...                                                ...   
138         138  [5143963, 5143964, 5143965, 5143966, 5143967, ...   
139         139  [5144521, 5144522, 5144523, 5144524, 5144525, ...   
140         140  [5182096, 5182097, 5182098, 5182099, 5182100, ...   
141         141  [5182171, 5182172, 5182173, 5182174, 5182175, ...   
142         142  [5182414, 5182415, 5182416, 5182417, 5182418, ...   

                                             READING_1  \
0    [-1, -1, -1, -1, -3, 0, -1, -3, -1, 1, -1, -1,...   
1    [-2, -2, -1,

In [48]:
# Create a connection to the SQLite database
conn = sqlite3.connect(db_file_path)

# Read USER_SESSION_TABLE to get STARTDATE for each SESSION_ID
user_session_query = 'SELECT SESSION_ID, START_DATE FROM USER_SESSION_TABLE'
user_session_df = pd.read_sql(user_session_query, conn)

# Merge user_session_df with grouped_right_insole and grouped_left_insole
merged_right_insole = pd.merge(grouped_right_insole, user_session_df, on='SESSION_ID', how='inner')
merged_left_insole = pd.merge(grouped_left_insole, user_session_df, on='SESSION_ID', how='inner')

# Close the database connection
conn.close()

# Specify the range of SESSION_ID values
start_session_id = 13
end_session_id = 106

# Root folder path
root_folder_path = 'database extractor/data/mci006/'

# Filter data for the specified SESSION_ID range
filtered_right_insole = merged_right_insole[(merged_right_insole['SESSION_ID'] >= start_session_id) & (merged_right_insole['SESSION_ID'] <= end_session_id)]
filtered_left_insole = merged_left_insole[(merged_left_insole['SESSION_ID'] >= start_session_id) & (merged_left_insole['SESSION_ID'] <= end_session_id)]

# Iterate through each row and save data as TXT in the specified folder
for index, session in filtered_right_insole.iterrows():
    session_id = session['SESSION_ID']
    start_date = str(session['START_DATE'])
    date_folder = start_date[:4] + '-' + start_date[4:6] + '-' + start_date[6:]
    array_data = session.drop(['SESSION_ID', 'START_DATE', 'RAW_ID'])
    session_df = pd.DataFrame(array_data).T
    folder_path = os.path.join(root_folder_path, date_folder)
    
    # Convert each array to pandas Series
    series_list = {col: pd.Series(data) for col, data in array_data.items()}
    if len(series_list['READING_1']) < 2 / 0.05: #! data can be less than 2 seconds, which is not necessary
        print(f'Session {session_id} less than 2 seconds, Skip')
        print("-----------------------------------------------------------")
        continue
    # Create a new DataFrame by concatenating the Series
    session_df = pd.DataFrame(series_list)
    
    os.makedirs(folder_path, exist_ok=True)  # Create folder if it doesn't exist
    filename = os.path.join(folder_path, f'S{session_id}_{start_date}_rawDataRight_{subject}.txt')
    session_df.to_csv(filename, sep=',', header=False, index=False)

for index, session in filtered_left_insole.iterrows():
    session_id = session['SESSION_ID']
    start_date = str(session['START_DATE'])
    date_folder = start_date[:4] + '-' + start_date[4:6] + '-' + start_date[6:]
    array_data = session.drop(['SESSION_ID', 'START_DATE', 'RAW_ID'])
    session_df = pd.DataFrame(array_data).T
    folder_path = os.path.join(root_folder_path, date_folder)
    
    # Convert each array to pandas Series
    series_list = {col: pd.Series(data) for col, data in array_data.items()}
    if len(series_list['READING_1']) < 2 / 0.05: #! data can be less than 2 seconds, which is not necessary
        print(f'Session {session_id} less than 2 seconds, Skip')
        print("-----------------------------------------------------------")
        continue
    # Create a new DataFrame by concatenating the Series
    session_df = pd.DataFrame(series_list)
    
    os.makedirs(folder_path, exist_ok=True)  # Create folder if it doesn't exist
    filename = os.path.join(folder_path, f'S{session_id}_{start_date}_rawDataLeft_{subject}.txt')
    session_df.to_csv(filename, sep=',', header=False, index=False)

Session 38 less than 2 seconds, Skip
-----------------------------------------------------------
Session 44 less than 2 seconds, Skip
-----------------------------------------------------------
Session 46 less than 2 seconds, Skip
-----------------------------------------------------------
Session 54 less than 2 seconds, Skip
-----------------------------------------------------------
Session 71 less than 2 seconds, Skip
-----------------------------------------------------------
Session 73 less than 2 seconds, Skip
-----------------------------------------------------------
Session 78 less than 2 seconds, Skip
-----------------------------------------------------------
Session 96 less than 2 seconds, Skip
-----------------------------------------------------------
Session 38 less than 2 seconds, Skip
-----------------------------------------------------------
Session 44 less than 2 seconds, Skip
-----------------------------------------------------------
Session 46 less than 2 seconds

In [46]:
len(series_list['READING_1'])

71628